In [38]:
import pymysql
import pandas as pd
import duckdb # pip install duckdb
from scipy.stats import chi2_contingency
from scipy import stats

In [39]:
dbConn = pymysql.connect(user='root', passwd='cksdyd123', host='127.0.0.1', db='Insu', charset='utf8')
cursor = dbConn.cursor(pymysql.cursors.DictCursor)

In [40]:
# MySQL에 쿼리하고 결과를 dataframe으로 반환
def sqldf(sql):
    cursor.execute(sql)
    result = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    
    # DataFrame 생성 시 열 이름 지정
    return pd.DataFrame(result, columns=column_names)
    # return pd.DataFrame(result)

# MySQL에 쿼리하고 결과를 딕셔너리로 반환
def sqldic(sql):
    cursor.execute(sql)
    return cursor.fetchall()

# 데이터프레임에 쿼리한 결과를 데이터프레임으로 반환
def dfsql(query):
   return duckdb.query(query).df()

In [41]:
def chi2(col, table):     
    keys = table['SIU_CUST_YN'].unique()
    d = {key: [] for key in keys}
    
    # 각 SIU_CUST_YN 별로 cnt 값을 수집합니다.
    for key in keys:
        counts = table[table['SIU_CUST_YN'] == key]['cnt'].tolist()
        d[key] = counts

    # 각 SIU_CUST_YN에 대해 cnt가 없으면 0으로 채웁니다.
    max_length = max(len(lst) for lst in d.values())
    for key in d:
        while len(d[key]) < max_length:
            d[key].append(0)  # 0으로 채움

    # DataFrame 생성
    table_df = pd.DataFrame(d)
    table_df.index = table[col].unique()  # 인덱스를 설정합니다.
    print(table_df) 
    return chi2_contingency(table_df)
def ttest(col, data):
    return stats.ttest_ind(data[col][data['SIU_CUST_YN']=='N'], data[col][data['SIU_CUST_YN']=='Y'], equal_var= False)


In [42]:
#거주시와 보험사기자는 관련이 있는가
table = sqldf(''' select  dense_rank() over (order by ctpr) as number,ctpr, SIU_CUST_YN,count(*) as cnt from Cust group by ctpr,SIU_CUST_YN order by ctpr,SIU_CUST_YN''')
table1= sqldf('''select  dense_rank() over (order by ctpr) as number, SIU_CUST_YN,count(*) as cnt from Cust group by ctpr,SIU_CUST_YN order by ctpr,SIU_CUST_YN''')
table

,number,ctpr,SIU_CUST_YN,cnt
0,1,강원,N,448
1,1,강원,Y,72
2,2,경기,N,4460
3,2,경기,Y,348
4,3,경남,N,1364
5,3,경남,Y,94
6,4,경북,N,843
7,4,경북,Y,45
8,5,광주,N,639
9,5,광주,Y,153


In [43]:
chi2('number', table1)

       N    Y
1    448   72
2   4460  348
3   1364   94
4    843   45
5    639  153
6    850   56
7    424   40
8   1191  198
9   3072  305
10    33    2
11   623   19
12  1150  180
13   932  102
14   788  102
15   174   39
16   653   15
17   553    0


Chi2ContingencyResult(statistic=np.float64(409.69883717208165), pvalue=np.float64(3.3702615176303684e-77), dof=16, expected_freq=array([[4.73903942e+02, 4.60960585e+01],
       [4.38178875e+03, 4.26211249e+02],
       [1.32875374e+03, 1.29246256e+02],
       [8.09282115e+02, 7.87178845e+01],
       [7.21792157e+02, 7.02078429e+01],
       [8.25686483e+02, 8.03135173e+01],
       [4.22868132e+02, 4.11318676e+01],
       [1.26587034e+03, 1.23129664e+02],
       [3.07764156e+03, 2.99358441e+02],
       [3.18973807e+01, 3.10261932e+00],
       [5.85089097e+02, 5.69109030e+01],
       [1.21210047e+03, 1.17899534e+02],
       [9.42339761e+02, 9.16602394e+01],
       [8.11104823e+02, 7.88951770e+01],
       [1.94118345e+02, 1.88816547e+01],
       [6.08784294e+02, 5.92157059e+01],
       [5.03978615e+02, 4.90213853e+01]]))

In [44]:
#결혼과 보험사기자 관련이 있는가
table = sqldf(''' select  dense_rank() over (order by WEDD_YN) as number,WEDD_YN, SIU_CUST_YN,count(*) as cnt from Cust group by WEDD_YN,SIU_CUST_YN order by WEDD_YN,SIU_CUST_YN''')
table1= sqldf(''' select  dense_rank() over (order by WEDD_YN) as number, SIU_CUST_YN,count(*) as cnt from Cust group by WEDD_YN,SIU_CUST_YN order by WEDD_YN,SIU_CUST_YN''')
table

,number,WEDD_YN,SIU_CUST_YN,cnt
0,1,N,N,8276
1,1,N,Y,713
2,2,Y,N,9921
3,2,Y,Y,1057


In [45]:
chi2('number', table1)

      N     Y
1  8276   713
2  9921  1057


Chi2ContingencyResult(statistic=np.float64(17.396072251781963), pvalue=np.float64(3.034523979939729e-05), dof=1, expected_freq=array([[ 8192.15871187,   796.84128813],
       [10004.84128813,   973.15871187]]))

In [46]:
#결혼한 사람의 자녀 여부는 보험사기자 관련이 있는가
table = sqldf(''' select  dense_rank() over (order by CHLD_CNT) as number,CHLD_CNT, SIU_CUST_YN,count(*) as cnt from Cust where WEDD_YN='Y' group by CHLD_CNT,SIU_CUST_YN order by CHLD_CNT,SIU_CUST_YN''')
table1= sqldf('''select  dense_rank() over (order by CHLD_CNT) as number, SIU_CUST_YN,count(*) as cnt from Cust  where WEDD_YN='Y' group by CHLD_CNT,SIU_CUST_YN having count(*)>1 order by CHLD_CNT,SIU_CUST_YN''')
table

,number,CHLD_CNT,SIU_CUST_YN,cnt
0,1,0,N,2465
1,1,0,Y,220
2,2,1,N,2933
3,2,1,Y,354
4,3,2,N,3770
5,3,2,Y,372
6,4,3,N,675
7,4,3,Y,86
8,5,4,N,75
9,5,4,Y,24


In [47]:
chi2('number', table1)

      N    Y
1  2465  220
2  2933  354
3  3770  372
4   675   86
5    75   24
6     3    0


Chi2ContingencyResult(statistic=np.float64(40.36174798809391), pvalue=np.float64(1.2623941313598213e-07), dof=5, expected_freq=array([[2.42669992e+03, 2.58300082e+02],
       [2.97078683e+03, 3.16213173e+02],
       [3.74353485e+03, 3.98465154e+02],
       [6.87790926e+02, 7.32090735e+01],
       [8.94760864e+01, 9.52391364e+00],
       [2.71139656e+00, 2.88603444e-01]]))

In [48]:
#계약상태코드와 보험사기자와 관련이 있는가
table = sqldf(''' select  dense_rank() over (order by CNTT_STAT_CODE) as number,CNTT_STAT_CODE, SIU_CUST_YN,count(*) as cnt from Cust c1,cntt c2 where c1.cust_id=c2.cust_id  group by CNTT_STAT_CODE,SIU_CUST_YN order by CNTT_STAT_CODE,SIU_CUST_YN''')
table1= sqldf(''' select  dense_rank() over (order by CNTT_STAT_CODE) as number, SIU_CUST_YN,count(*) as cnt from Cust c1,cntt c2 where c1.cust_id=c2.cust_id  group by CNTT_STAT_CODE,SIU_CUST_YN order by CNTT_STAT_CODE,SIU_CUST_YN''')
table

,number,CNTT_STAT_CODE,SIU_CUST_YN,cnt
0,1,1,N,19762
1,1,1,Y,1486
2,2,2,N,2840
3,2,2,Y,174
4,3,3,N,302
5,3,3,Y,56
6,4,4,N,1057
7,4,4,Y,382
8,5,9,N,2912
9,5,9,Y,157


In [49]:
chi2('number', table1)

        N     Y
1   19762  1486
2    2840   174
3     302    56
4    1057   382
5    2912   157
6     302    27
7   19666  2582
8      57    13
9       5     1
10   1111   248
11     10     3
12    113     8
13   1215   127
14   2756   429
15    481    58


Chi2ContingencyResult(statistic=np.float64(1037.8493757210879), pvalue=np.float64(1.1808803702974564e-212), dof=14, expected_freq=array([[1.91534294e+04, 2.09457059e+03],
       [2.71688800e+03, 2.97111999e+02],
       [3.22709325e+02, 3.52906754e+01],
       [1.29714726e+03, 1.41852743e+02],
       [2.76646625e+03, 3.02533750e+02],
       [2.96568067e+02, 3.24319335e+01],
       [2.00548521e+04, 2.19314789e+03],
       [6.30995886e+01, 6.90041138e+00],
       [5.40853617e+00, 5.91463833e-01],
       [1.22503344e+03, 1.33966558e+02],
       [1.17184950e+01, 1.28150497e+00],
       [1.09072146e+02, 1.19278540e+01],
       [1.20970926e+03, 1.32290744e+02],
       [2.87103128e+03, 3.13968718e+02],
       [4.85866832e+02, 5.31331676e+01]]))

In [50]:
#주계약의 보험금액과 보험사기자 분석
data = sqldf(''' select SIU_CUST_YN, MAIN_INSR_AMT from Cust c, Cntt n where c.cust_id = n.cust_id ''')
dfsql('''select SIU_CUST_YN, AVG(MAIN_INSR_AMT) from data group by SIU_CUST_YN ''')

,SIU_CUST_YN,avg(MAIN_INSR_AMT)
0,Y,3.824018e+07
1,N,3.389920e+07


In [51]:
ttest('MAIN_INSR_AMT', data)

TtestResult(statistic=np.float64(-7.028218300045751), pvalue=np.float64(2.3040809599701305e-12), df=np.float64(6533.964800922198))

In [52]:
#주계약의 보험금액과 보험사기자 추가분석
data = sqldf(''' select SIU_CUST_YN, MAIN_INSR_AMT from Cust c, Cntt n where c.cust_id = n.cust_id and CNTT_STAT_CODE IN ('3','4','A','B','J','L') ''')
dfsql('''select SIU_CUST_YN, AVG(MAIN_INSR_AMT) from data group by SIU_CUST_YN ''')

,SIU_CUST_YN,avg(MAIN_INSR_AMT)
0,Y,3.906024e+07
1,N,3.564091e+07


In [53]:
ttest('MAIN_INSR_AMT', data)

TtestResult(statistic=np.float64(-3.8878596350488026), pvalue=np.float64(0.00010270936203514208), df=np.float64(4158.980469926344))

In [54]:
#유의병원 여부와  보험사기자와 관련이 있는가
table = sqldf(''' select  dense_rank() over (order by HEED_HOSP_YN) as number,HEED_HOSP_YN, SIU_CUST_YN,count(*) as cnt from Cust c1,claim c2 where c1.cust_id=c2.cust_id  group by HEED_HOSP_YN,SIU_CUST_YN order by HEED_HOSP_YN,SIU_CUST_YN''')
table1 = sqldf(''' select  dense_rank() over (order by HEED_HOSP_YN) as number, SIU_CUST_YN,count(*) as cnt from Cust c1,claim c2 where c1.cust_id=c2.cust_id  group by HEED_HOSP_YN,SIU_CUST_YN order by HEED_HOSP_YN,SIU_CUST_YN''')
table

,number,HEED_HOSP_YN,SIU_CUST_YN,cnt
0,1,N,N,6747
1,1,N,Y,416
2,2,Y,N,153
3,2,Y,Y,7


In [55]:
chi2('number', table1)  

      N    Y
1  6747  416
2   153    7


Chi2ContingencyResult(statistic=np.float64(0.35629997856063217), pvalue=np.float64(0.5505683482591366), dof=1, expected_freq=array([[6749.24211389,  413.75788611],
       [ 150.75788611,    9.24211389]]))

In [56]:
#병원 지역이 보험사기자와 관련이 있는가
table = sqldf(''' select  dense_rank() over (order by ACCI_HOSP_ADDR) as number,ACCI_HOSP_ADDR, SIU_CUST_YN,count(*) as cnt from Cust c1,claim c2 where c1.cust_id=c2.cust_id and ACCI_HOSP_ADDR not in('None','세종','충북')  group by ACCI_HOSP_ADDR,SIU_CUST_YN order by ACCI_HOSP_ADDR,SIU_CUST_YN''')
table1 = sqldf(''' select  dense_rank() over (order by ACCI_HOSP_ADDR) as number, SIU_CUST_YN,count(*) as cnt from Cust c1,claim c2 where c1.cust_id=c2.cust_id and ACCI_HOSP_ADDR not in('None','세종','충북')  group by ACCI_HOSP_ADDR,SIU_CUST_YN order by ACCI_HOSP_ADDR,SIU_CUST_YN''')
table

,number,ACCI_HOSP_ADDR,SIU_CUST_YN,cnt
0,1,강원,N,144
1,1,강원,Y,10
2,2,경기,N,1669
3,2,경기,Y,60
4,3,경남,N,374
5,3,경남,Y,5
6,4,경북,N,199
7,4,경북,Y,8
8,5,광주,N,204
9,5,광주,Y,16


In [57]:
chi2('number', table1)  

       N   Y
1    144  10
2   1669  60
3    374   5
4    199   8
5    204  16
6    394  35
7    214  29
8    502  36
9   1801  81
10   242   2
11   342  11
12   204   8
13   119  91
14    63   1
15   197  12


Chi2ContingencyResult(statistic=np.float64(631.5234965518922), pvalue=np.float64(1.0316787636628581e-125), dof=14, expected_freq=array([[ 145.18195956,    8.81804044],
       [1629.99745511,   99.00254489],
       [ 357.29845893,   21.70154107],
       [ 195.14717941,   11.85282059],
       [ 207.40279938,   12.59720062],
       [ 404.43545879,   24.56454121],
       [ 229.08581931,   13.91418069],
       [ 507.19411848,   30.80588152],
       [1774.23667468,  107.76332532],
       [ 230.02855931,   13.97144069],
       [ 332.787219  ,   20.212781  ],
       [ 199.8608794 ,   12.1391206 ],
       [ 197.97539941,   12.02460059],
       [  60.33535982,    3.66464018],
       [ 197.03265941,   11.96734059]]))

In [58]:
#병원 종별 구분이 보험사기자와 관련이 있는가
table = sqldf(''' select  dense_rank() over (order by HOSP_SPEC_DVSN) as number,HOSP_SPEC_DVSN, SIU_CUST_YN,count(*) as cnt from Cust c1,claim c2 where c1.cust_id=c2.cust_id  group by HOSP_SPEC_DVSN,SIU_CUST_YN order by HOSP_SPEC_DVSN,SIU_CUST_YN''')
table1 = sqldf(''' select  dense_rank() over (order by HOSP_SPEC_DVSN) as number, SIU_CUST_YN,count(*) as cnt from Cust c1,claim c2 where c1.cust_id=c2.cust_id  group by HOSP_SPEC_DVSN,SIU_CUST_YN order by HOSP_SPEC_DVSN,SIU_CUST_YN''')
table

,number,HOSP_SPEC_DVSN,SIU_CUST_YN,cnt
0,1,0,N,4
1,1,0,Y,3
2,2,10,N,3165
3,2,10,Y,101
4,3,20,N,1898
5,3,20,Y,134
6,4,25,N,22
7,4,25,Y,8
8,5,30,N,1497
9,5,30,Y,101


In [59]:
chi2('number', table1)  

       N    Y
1      4    3
2   3165  101
3   1898  134
4     22    8
5   1497  101
6     34    1
7     63    2
8     82   42
9     49   16
10    86   15


Chi2ContingencyResult(statistic=np.float64(327.24648075212843), pvalue=np.float64(4.280306642244178e-65), dof=9, expected_freq=array([[6.59565752e+00, 4.04342483e-01],
       [3.07734535e+03, 1.88654650e+02],
       [1.91462515e+03, 1.17374846e+02],
       [2.82671036e+01, 1.73289635e+00],
       [1.50569439e+03, 9.23056125e+01],
       [3.29782876e+01, 2.02171241e+00],
       [6.12453912e+01, 3.75460877e+00],
       [1.16837362e+02, 7.16263826e+00],
       [6.12453912e+01, 3.75460877e+00],
       [9.51659156e+01, 5.83408439e+00]]))

In [60]:
#병원거리와 보험사기자는 관련이 있는가
data = sqldf(''' select SIU_CUST_YN, HOUSE_HOSP_DIST from Cust c,claim n where c.cust_id = n.cust_id ''')
dfsql('''select SIU_CUST_YN, AVG(HOUSE_HOSP_DIST) from data group by SIU_CUST_YN ''')


,SIU_CUST_YN,avg(HOUSE_HOSP_DIST)
0,N,22.919913
1,Y,22.194397


In [61]:
ttest('HOUSE_HOSP_DIST', data)

TtestResult(statistic=np.float64(0.23462544842867875), pvalue=np.float64(0.8146022763531788), df=np.float64(467.0603563973655))

In [107]:
#강원 전북에 있는 병원거리와 보험사기자는 관련이 있는가
data = sqldf(''' select SIU_CUST_YN, HOUSE_HOSP_DIST from Cust c,claim n where c.cust_id = n.cust_id and ACCI_HOSP_ADDR in ('강원','전북') ''')
dfsql('''select SIU_CUST_YN, AVG(HOUSE_HOSP_DIST) from data group by SIU_CUST_YN ''')


,SIU_CUST_YN,avg(HOUSE_HOSP_DIST)
0,N,29.954106
1,Y,13.106535


In [63]:
ttest('HOUSE_HOSP_DIST', data)

TtestResult(statistic=np.float64(2.72368666085682), pvalue=np.float64(0.007135679779025698), df=np.float64(168.6182046127902))

In [66]:
#청구사유코드와 보험사기자는 관련이 있는가
table = sqldf(''' select  dense_rank() over (order by DMND_RESN_CODE) as number,DMND_RESN_CODE, SIU_CUST_YN,count(*) as cnt from Cust c1,claim c2 where c1.cust_id=c2.cust_id  group by DMND_RESN_CODE,SIU_CUST_YN order by DMND_RESN_CODE,SIU_CUST_YN''')
table1 = sqldf(''' select  dense_rank() over (order by DMND_RESN_CODE) as number, SIU_CUST_YN,count(*) as cnt from Cust c1,claim c2 where c1.cust_id=c2.cust_id  group by DMND_RESN_CODE,SIU_CUST_YN order by DMND_RESN_CODE,SIU_CUST_YN''')
table

,number,DMND_RESN_CODE,SIU_CUST_YN,cnt
0,1,2,N,3228
1,1,2,Y,319
2,2,3,N,3672
3,2,3,Y,104


In [67]:
chi2('number', table1)  

      N    Y
1  3228  319
2  3672  104


Chi2ContingencyResult(statistic=np.float64(129.67232944376588), pvalue=np.float64(4.833268368667638e-30), dof=1, expected_freq=array([[3342.11388775,  204.88611225],
       [3557.88611225,  218.11388775]]))

In [ ]:
#직업그룹코드와 보험사기자는 관련이 있는가
table = sqldf(''' select  dense_rank() over (order by ACCI_OCCP_GRP2) as number,ACCI_OCCP_GRP2, SIU_CUST_YN,count(*) as cnt from Cust c1,claim c2 where c1.cust_id=c2.cust_id and ACCI_OCCP_GRP2 Not in('고소득 전문직', '고소득의료직','고위 공무원','교사','기업/단체 임원', '기타','대학교수/강사'
              ,'법무직 종사자', '전문직', '학자/연구직') group by ACCI_OCCP_GRP2,SIU_CUST_YN order by ACCI_OCCP_GRP2,SIU_CUST_YN''')
table1 = sqldf(''' select  dense_rank() over (order by ACCI_OCCP_GRP2) as number, SIU_CUST_YN,count(*) as cnt from Cust c1,claim c2 where c1.cust_id=c2.cust_id and ACCI_OCCP_GRP2 Not in('고소득 전문직', '고소득의료직','고위 공무원','교사','기업/단체 임원', '기타','대학교수/강사'
              ,'법무직 종사자', '전문직', '학자/연구직') group by ACCI_OCCP_GRP2,SIU_CUST_YN order by ACCI_OCCP_GRP2,SIU_CUST_YN''')
table

,number,ACCI_OCCP_GRP2,SIU_CUST_YN,cnt
0,1,1차산업 종사자,N,100
1,1,1차산업 종사자,Y,4
2,2,2차산업 종사자,N,647
3,2,2차산업 종사자,Y,10
4,3,3차산업 종사자,N,893
5,3,3차산업 종사자,Y,63
6,4,공무원,N,111
7,4,공무원,Y,10
8,5,교육관련직,N,160
9,5,교육관련직,Y,11


In [77]:
chi2('number', table1)  

       N    Y
1    100    4
2    647   10
3    893   63
4    111   10
5    160   11
6     66   13
7     93    5
8    258   39
9    619   48
10    54    3
11   138    4
12   104    2
13   539   59
14    64    1
15  1234  109
16  1529   42


Chi2ContingencyResult(statistic=np.float64(134.97558631269146), pvalue=np.float64(2.2464928415958055e-21), dof=15, expected_freq=array([[  97.7440273 ,    6.2559727 ],
       [ 617.47909556,   39.52090444],
       [ 898.49317406,   57.50682594],
       [ 113.72141638,    7.27858362],
       [ 160.7137372 ,   10.2862628 ],
       [  74.24786689,    4.75213311],
       [  92.10494881,    5.89505119],
       [ 279.13438567,   17.86561433],
       [ 626.87755973,   40.12244027],
       [  53.57124573,    3.42875427],
       [ 133.45819113,    8.54180887],
       [  99.62372014,    6.37627986],
       [ 562.028157  ,   35.971843  ],
       [  61.09001706,    3.90998294],
       [1262.2137372 ,   80.7862628 ],
       [1476.49872014,   94.50127986]]))

In [ ]:
#(사고접수-원사고)와 보험사기자 관련이 있는가
data = sqldf(''' select SIU_CUST_YN, DATEDIFF(n.RECP_DATE, n.ORIG_RESN_DATE) date from Cust c,claim n where c.cust_id = n.cust_id ''')
dfsql('''select SIU_CUST_YN, AVG(date) from data group by SIU_CUST_YN ''')


,SIU_CUST_YN,avg(date)
0,N,151.789565
1,Y,185.917258


In [125]:
ttest('date', data)

TtestResult(statistic=np.float64(-2.3433020567974236), pvalue=np.float64(0.019533608667740766), df=np.float64(465.8271803029734))

In [139]:
#(사고접수-원사고)와 보험사기자 관련이 있는가
data = sqldf(''' select SIU_CUST_YN, DMND_AMT from Cust c,claim n where c.cust_id = n.cust_id and DMND_AMT >SELF_CHAM ''')
dfsql('''select SIU_CUST_YN, AVG(DMND_AMT) from data group by SIU_CUST_YN ''')

,SIU_CUST_YN,avg(DMND_AMT)
0,Y,739120.354730
1,N,566266.772356


In [140]:
ttest('DMND_AMT', data)

TtestResult(statistic=np.float64(-2.191720858794243), pvalue=np.float64(0.029147525772331793), df=np.float64(308.24118481791345))